Reference [Peter Norvig](https://norvig.com/lispy.html)

Let's just make a simple calculator.  We want to be able to use it like:
```
(define r 10)
(* pi (* r r))
```

```
>> program = "(begin (define r 10) (* pi (* r r)))"

>>> parse(program)
['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

>>> eval(parse(program))
314.1592653589793
```

## Type Definitions

In [1]:
Symbol = str              # Symbol is implemented as a Python str
Number = (int, float)     # Number is implemented as either a Python int or float
Atom   = (Symbol, Number) # An Atom is a Symbol or Number
List   = list             # List is implemented as a Python list
Exp    = (Atom, List)     # An expression is either an Atom or List
Env    = dict             # An environment is a mapping of {variable: value}. Dict for now; we'll expand later.

In [2]:
def tokenize(chars: str) -> list:
    """
    Convert a string of characters into a list of tokens.
    """
    return chars.replace('(', ' ( ').replace(')', ' ) ').split()

In [3]:
program = "(begin (define r 10) (* pi (* r r)))"
print(tokenize(program))

['(', 'begin', '(', 'define', 'r', '10', ')', '(', '*', 'pi', '(', '*', 'r', 'r', ')', ')', ')']


In [4]:
def parse(program: str) -> Exp:
    """
    Read a string and turn it into an Expression.
    """
    return read_from_tokens(tokenize(program))

def read_from_tokens(tokens: list) -> Exp:
    """
    Read an expression from a sequence of tokens.
    """
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF')
    
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0) # pop off ')'
        return L
    
    if token == ')':
        raise SyntaxError('unexpected )')
        
    return atom(token)

def atom(token: str) -> Atom:
    """
    Numbers remain numbers; every other token becomes a symbol.
    """
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return Symbol(token)

In [12]:
program

'(begin (define r 10) (* pi (* r r)))'

In [13]:
parse(program)

['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

## Environments

In [14]:
#*TODO: make this a class
#*TODO: consider other ways to update the global space and expand it by importing modules
import math
import operator as op

def standard_env() -> Env:
    "An environment with some Scheme standard procedures."
    env = Env()
    env.update(vars(math)) # sin, cos, sqrt, pi, ...
    env.update({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '=':op.eq, 
        'abs':     abs,
        'append':  op.add,  
        'apply':   lambda proc, args: proc(*args),
        'begin':   lambda *x: x[-1],
        'car':     lambda x: x[0],
        'cdr':     lambda x: x[1:], 
        'cons':    lambda x,y: [x] + y,
        'eq?':     op.is_, 
        'expt':    pow,
        'equal?':  op.eq, 
        'length':  len, 
        'list':    lambda *x: List(x), 
        'list?':   lambda x: isinstance(x, List), 
        'map':     map,
        'max':     max,
        'min':     min,
        'not':     op.not_,
        'null?':   lambda x: x == [], 
        'number?': lambda x: isinstance(x, Number),  
        'print':   print,
        'procedure?': callable,
        'round':   round,
        'symbol?': lambda x: isinstance(x, Symbol),
    })
    return env

global_env = standard_env()

## Evaluation: eval

In [15]:
def eval(x: Exp, env=global_env) -> Exp:
    """
    Evaluate an expression in an environment.
    """
    if isinstance(x, Symbol):        # variable reference
        return env[x]
    if isinstance(x, Number):      # constant number
        return x                
    if x[0] == 'if':               # conditional
        (_, test, conseq, alt) = x
        result = eval(test,env)
        exp = (conseq if eval(test, env) else alt)
        return eval(exp, env)
    if x[0] == 'define':           # definition
        (_, symbol, exp) = x
        env[symbol] = eval(exp, env)
        return None
    
    # Procedure call.
    proc = eval(x[0], env)
    args = [eval(arg, env) for arg in x[1:]]
    return proc(*args)

In [16]:
eval(parse(program))

314.1592653589793

In [17]:
def run(program: str) -> Exp:
    return eval(parse(program))

In [18]:
print(program)

(begin (define r 10) (* pi (* r r)))


In [19]:
run(program)

314.1592653589793

## Interaction: REPL

In [20]:
def repl(prompt='> '):
    """
    A prompt-read-eval-print loop.
    """
    while True:
        text = input(prompt)
        if text == 'exit':
            break
        val = eval(parse(text))
        if val is not None: 
            print(unparse(val))

def unparse(exp):
    """
    Convert an expression's internal representation Python object back into a parsable string.
    """
    if isinstance(exp, List):
        return '(' + ' '.join(map(unparse, exp)) + ')' 
    else:
        return str(exp)

Try this:
```
repl()
(+ 1 2)
exit
```

In [21]:
repl()

> (+ 1 2)
3
> exit


Try this:
```
repl()
> (define r 10)
> (* pi (* r r))
314.1592653589793
> (if (> (* 11 11) 12) (* 7 6) oops)
```

In [41]:
run('(if (> 1 2) 1 0)')

0

In [48]:
run("""
    (if (> (* 11 11) 12) (* 7 6) oops)
""")

42

In [46]:
run("""
    (list (+ 1 1) (+ 2 2) (* 2 3) (expt 2 3))
""")

[2, 4, 6, 8]

### Make Env a Class

In [22]:
class Env(dict):
    """
    An environment is a dict of name value pairs, with an outer Env.
    """
    def __init__(self, parms=(), args=(), outer: Env=None):
        """
        @param parms: Variable names to bind arguments to.
        @param args: Values to bind the variable names to.
        """
        self.update(zip(parms, args))
        self.outer = outer
    
    def find(self, var: str) -> Env:
        """
        Finds the innermost Env where the given name appears.
        @param var: The name of the variable we're looking for.
        @returns Env: The environment in which this name appears.
        """
        if var in self:
            return self
        return self.outer.find(var)

In [23]:
# change global_env over to the new Env class.
global_env = standard_env()

## User-defined procedures

In [24]:
class Procedure:
    """
    A user-defined procedure with variable name bindings.
    """
    def __init__(self, parms, body, env):
        self.parms = parms
        self.body = body
        self.env = env
    
    def __call__(self, *args):
        # Create an environment with bindings for this one invocation.
        env = Env(self.parms, args, self.env)

        return eval(self.body, env)        

**TODO**: Figure out how to enable plugins into `eval()`

In [25]:
def eval(x: Exp, env=global_env) -> Exp:
    """
    Evaluate an expression in an environment.
    """
    if isinstance(x, Symbol):        # variable reference
        return env.find(x)[x]

    if not isinstance(x, List):      # constant number
        return x
    
    op, *args = x
    if op == 'quote':              # Quote an expression without evaluating it
        return args[0]
    
    if op == 'if':               # conditional
        (test, conseq, alt) = args
        result = eval(test,env)
        if result:
            exp = conseq
        else:
            exp = alt
        return eval(exp, env)

    if op == 'define':           # definition
        (symbol, exp) = args
        env[symbol] = eval(exp, env)
        return None

    if op == 'set!':             # assignment
        (symbol, exp) = args
        env.find(symbol)[symbol] = eval(exp, env)
        return None
    
    if op == 'lambda':           # procedure
        (parms, body) = args
        return Procedure(parms, body, env)
        
    # Procedure call.
    proc = eval(x[0], env)
    args = [eval(arg, env) for arg in x[1:]]
    return proc(*args)

In [60]:
run("""
    (define make-account
        (lambda (balance)
            (lambda (amt)
                (begin (set! balance (+ balance amt))
                        balance))))
    (define account1 (make-account 100))
    (account1 -20)
""")

define args  ['make-account', ['lambda', ['balance'], ['lambda', ['amt'], ['begin', ['set!', 'balance', ['+', 'balance', 'amt']], 'balance']]]]


```
(define make-account (lambda (balance) (lambda (amt) (begin (set! balance (+ balance amt)) balance))))


(define account1 (make-account 100.00))
(account1 -20.00)
(define account1 (make-account 100.00))
(account1 -20.00)
(account1 -20.00)
(define account2 (make-account 100.00))
(account2 40)
(account2 -10)
```

In [68]:
repl()

> (define make-account (lambda (balance) (lambda (amt) (begin (set! balance (+ balance amt)) balance))))
> (define account1 (make-account 100.00))
> account1
> (account1 -20.00)
80.0
> (account1 -20.00)
60.0
> (define account2 (make-account 100.00))
> (account2 40)
140.0
> (account2 -10)
130.0
> exit


## Next steps

First let's assemble a list of things we'd like to do.  Our goal is to write something non-trivial like Asteroids.  To do that there are a few things we need to add.

### Must Have
* [Tail Call Optimization](https://en.wikipedia.org/wiki/Tail_call)
    * To write a simple game I need to iterate infinitely.  This means either TCO, or cheating by implementing an explicit loop structure and introduce a new keyword.
* [Data Structures](https://www.csie.ntu.edu.tw/~course/10420/Resources/lp/node50.html)
    * [Association Lists](https://www.csie.ntu.edu.tw/~course/10420/Resources/lp/node51.html)
* Cleanup/exception handling.
    * To write a game, I need to create graphics structures such as windows which need to be de-allocated if the game halts unexpectedly.
* Stack traces
    * We'll need to be able to report what the interpreter was doing when we, for example, reference a variable that doesn't exist.  To do that we'd probably change the parser to attach properties with each token detailing the file, line number, and character position.
* Macros
* Plugin/module architecture.
    * I don't like needing to update `eval()` when we add new structures.  Should be able to load a module that uses either lisp or python functions that hook into `eval()`.
* Interpreter object.  I'd like to have a top-level object to encapsulate a lot of these global variables.  This would also enable me to have multiple interpreters that don't interfere with each other.
    
Some of these will require pretty significant rework of the code, and it'll be easy to get it wrong and break it.  It's time to make unit tests.

What should our unit tests look like?  We could code them in Python, but let's code the above test in Lisp.

In [2]:
code = """
    (define make-account
        (lambda (balance)
            (lambda (amt)
                (begin (set! balance (+ balance amt))
                        balance))))
    (define account1 (make-account 100))
    (expect-equal 80.0 (account1 -20))
    (expect-equal 60.0 (account1 -20))
    
    (define account2 (make-account 100))
    (expect-equal 140.0 (account2 40))
    (expect-equal 130.0 (account2 -10))
"""

We will need to implement an `expect-equal` function somehow.  And also we'll want to be able to make many such tests.  Having the interpreter object will be a good first step, so that we have a good clean framework for running the tests instead of having dangling global variables.  Let's move over to an actual codebase outside of the notebooks.